In [22]:
import json
import numpy as np
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from operator import itemgetter 
import time


In [23]:
url = 'https://github.com/DigixGlobal'



In [37]:

            
def repolink_extract(url):
    
    html = requests.get(url)
    htmltxt = bs(html.text, 'html.parser')
    
    
    repohtml = htmltxt.find_all('a', class_="d-inline-block", 
                                itemprop="name codeRepository")

    partial_repolink_list = []
    for item in repohtml:
        link = item.attrs['href']
        partial_repolink_list.append(link)
    return partial_repolink_list
    

    

def nextpage_extract(url):
    html = requests.get(url)
    htmltxt = bs(html.text, 'html.parser')

    nextpage = htmltxt.find_all('a', 
                    class_="next_page",
                    rel="next")
    if len(nextpage) != 0:
        a = 'https://github.com'
        next_ = nextpage[0].attrs['href']
        nextlink = '{}{}'.format(a, next_)
        return nextlink
    else:
        return None


def repolink_extract_paginator(current_url):
    output = repolink_extract(current_url)
    next_url = nextpage_extract(current_url)
    while next_url is not None:
        current_url = next_url
        output_temp = repolink_extract(current_url)
        output.append(output_temp)
        next_url = nextpage_extract(current_url)
        time.sleep(1)
    repolink_list = []
    def reemoveNestings(l):
        for i in l:
            if type(i) == list:
                reemoveNestings(i)
            else:
                repolink_list.append(i)
    reemoveNestings(output)
    
    return repolink_list
    

In [44]:
def gitfeature_html(github_account_html):
    html = requests.get(github_account_html)
    htmltxt = bs(html.text, 'html.parser')
    repohtml = htmltxt.find_all('a', class_="text-bold flex-auto ")
    if len(repohtml) == 0:
        output = gitfeature_extract(github_account_html.replace('https://github.com', ''))
        
    
    else:
        repolink_extract_paginator(github_account_html)
        

def gitfeature_extract(partial_repolink):
    a = 'https://github.com'
    full_repolink = '{}{}'.format(a, partial_repolink)
    githtml = requests.get(full_repolink)
    githtmltxt = bs(githtml.text, 'html.parser')
    watch_html = githtmltxt.find_all(
        'a', 
        class_="social-count",
        href= '{}{}'.format(partial_repolink,'/watchers')
    )
    watch = watch_html[0].text.strip().replace(',', '')
    stars_html = githtmltxt.find_all(
        'a',
        class_="social-count",
        href= '{}{}'.format(partial_repolink,'/stargazers')
    )
    stars = stars_html[0].text.strip().replace(',', '')
    forks_html = githtmltxt.find_all(
        'a',
        href= '{}{}'.format(partial_repolink,'/network/members'),
        class_="social-count"
    )
    
    forks = forks_html[0].text.strip().replace(',', '')

    otherfeature_html = githtmltxt.find_all(
       'span', class_="num text-emphasized")
    commits = otherfeature_html[0].text.strip().replace(',', '')
    branches = otherfeature_html[1].text.strip().replace(',', '')
    releases = otherfeature_html[2].text.strip().replace(',', '')
    gitfeature = {
        'watchers':watch,
        'stars': stars,
        'forks':forks, 
        'commits':commits,
        'branches':branches}
    return gitfeature





In [45]:
def repolink_extract_paginator(current_url):
    output = repolink_extract(current_url)
    next_url = nextpage_extract(current_url)
    while next_url is not None:
        current_url = next_url
        output_temp = repolink_extract(current_url)
        output.append(output_temp)
        next_url = nextpage_extract(current_url)
        time.sleep(1)
    repolink_list = []
    def reemoveNestings(l):
        for i in l:
            if type(i) == list:
                reemoveNestings(i)
            else:
                repolink_list.append(i)
    reemoveNestings(output)
    d_list = []
    feature_sum = {}
    for partial_repolink in repolink_list:
        feature_dic = gitfeature_extract(partial_repolink)
        d_list.append(feature_dic)
    for d in d_list:
        for k in d.keys():
            feature_sum[k] = feature_sum.get(k, 0) + int((d[k]))
    return feature_sum

In [46]:
repolink_extract_paginator(url)

{'watchers': 318,
 'stars': 654,
 'forks': 20163,
 'commits': 29202,
 'branches': 707}

In [ ]:
url2 = 'https://github.com/AugurProject'

In [6]:
repolink_extract_paginator(url2)

['/AugurProject/augur',
 '/AugurProject/augur-ui',
 '/AugurProject/augur-node',
 '/AugurProject/landing-page',
 '/AugurProject/docs',
 '/AugurProject/augur-app',
 '/AugurProject/solidity-docgen',
 '/AugurProject/augur.js',
 '/AugurProject/solidity-typescript-generator',
 '/AugurProject/ethereum-nodes',
 '/AugurProject/augur-core',
 '/AugurProject/Augur-Node-Experiment',
 '/AugurProject/knex',
 '/AugurProject/whitepaper',
 '/AugurProject/ethereumjs-blockstream',
 '/AugurProject/speedomatic',
 '/AugurProject/augur-ui-extractor',
 '/AugurProject/augur-walkthrough',
 '/AugurProject/rep-migration',
 '/AugurProject/MyCrypto',
 '/AugurProject/etherwallet',
 '/AugurProject/eth-contract-metadata',
 '/AugurProject/augur-bounties',
 '/AugurProject/solidity-flattener',
 '/AugurProject/go-ethereum',
 '/AugurProject/branding',
 '/AugurProject/electron-metamask-boilerplate',
 '/AugurProject/bignumber.js',
 '/AugurProject/EIPs',
 '/AugurProject/react-blockies',
 '/AugurProject/pyethereum',
 '/AugurPro

In [35]:
current_url = 'https://github.com/DigixGlobal'
output = repolink_extract(current_url)
next_url = nextpage_extract(current_url)
while next_url is not None:
    current_url = next_url
    output_temp = repolink_extract(current_url)
    output.append(output_temp)
    next_url = nextpage_extract(current_url)
    time.sleep(1)
repolink_list = []
def reemoveNestings(l):
    for i in l:
        if type(i) == list:
            reemoveNestings(i)
        else:
            repolink_list.append(i)
reemoveNestings(output)
d_list = []
feature_sum = {}
for partial_repolink in repolink_list:
    feature_dic = gitfeature_extract(partial_repolink)
    d_list.append(feature_dic)

In [36]:
d_list

[{'watchers': '5',
  'stars': '0',
  'forks': '0',
  'commits': '35',
  'branches': '41'},
 {'watchers': '0',
  'stars': '0',
  'forks': '0',
  'commits': '8',
  'branches': '7'},
 {'watchers': '6',
  'stars': '0',
  'forks': '0',
  'commits': '2',
  'branches': '23'},
 {'watchers': '6',
  'stars': '0',
  'forks': '0',
  'commits': '619',
  'branches': '50'},
 {'watchers': '5',
  'stars': '0',
  'forks': '0',
  'commits': '5',
  'branches': '32'},
 {'watchers': '5',
  'stars': '2',
  'forks': '1',
  'commits': '14',
  'branches': '4'},
 {'watchers': '8',
  'stars': '6',
  'forks': '3',
  'commits': '23',
  'branches': '2'},
 {'watchers': '7',
  'stars': '22',
  'forks': '2',
  'commits': '77',
  'branches': '12'},
 {'watchers': '5',
  'stars': '0',
  'forks': '0',
  'commits': '13',
  'branches': '2'},
 {'watchers': '0',
  'stars': '0',
  'forks': '1',
  'commits': '8',
  'branches': '1'},
 {'watchers': '1',
  'stars': '0',
  'forks': '6',
  'commits': '11',
  'branches': '2'},
 {'watc

In [32]:
hrff = gitfeature_extract('/AugurProject/augur')

In [33]:
hrff

dict_items([('watchers', '13'), ('stars', '85'), ('forks', '25'), ('commits', '30,974'), ('branches', '66')])

In [43]:
int(float(1,194))

TypeError: float() takes at most 1 argument (2 given)